In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('books.db')

In [3]:
import pandas as pd

In [4]:
pd.options.display.max_columns = 12

In [5]:
pd.read_sql("SELECT * FROM authors", connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [7]:
#now we will check out the titles
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


In [9]:
#lets jsut loook at five rows
df = pd.read_sql('SELECT * FROM author_ISBN', connection)
df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


In [10]:
#just first andladst form authros table
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


In [13]:
#now we will use th ewhere clause 
pd.read_sql("""SELECT title, edition, copyright
                    FROM titles
                    WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


In [14]:
#using the perxcent symmbol, we will find strings that have zero or more charachters at the percent character's psoittion in the pattern
pd.read_sql("""SELECT id, first, last
                    FROM authors
                    WHERE last LIKE 'D%'""",
                 connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


In [15]:
#using the underscore we wil find a wildcard character in that position
pd.read_sql("""SELECT id, first, last
                    FROM authors
                    WHERE first LIKE '_b%'""",
                 connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


In [16]:
#using the order by clause, we will sort into ascending order
pd.read_sql('SELECT title FROM titles ORDER BY title ASC',
                  connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In [17]:
#sort using the ORDER BY 
pd.read_sql("""SELECT id, first, last
                    FROM authors
                    ORDER BY last, first""",
                   connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [18]:
#sorting by descending order by last name and asceniding order by first name for authors who have the same last name
pd.read_sql("""SELECT id, first, last
                    FROM authors
                    ORDER BY last DESC, first ASC""",
                  connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


In [19]:
#now we are going to use th eWHERE and the ORDER BY clauses. These will be comobined into one query
pd.read_sql("""SELECT isbn, title, edition, copyright
                    FROM titles
                    WHERE title LIKE '%How to Program'
                    ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


In [20]:
#now we will merge data using the INNER JOIN
pd.read_sql("""SELECT first, last, isbn
                    FROM authors
                    INNER JOIN author_ISBN
                        ON authors.id = author_ISBN.id
                    ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


In [21]:
#now we will start to mdoify the database. We wil lbegin by calliugn the cursor
cursor = connection.cursor()

In [22]:
#now we will insert a new author! Their name is Bobby Blue
cursor = cursor.execute("""INSERT INTO authors (first, last)
                                VALUES ('Bobby', 'Blue')""")

In [23]:
pd.read_sql('SELECT id, first, last FROM authors',
                 connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Bobby,Blue


In [24]:
#let us UPDATE their name!
cursor = cursor.execute("""UPDATE authors SET last='Gray'
                                WHERE last='Blue' AND first='Bobby'""")

In [25]:
pd.read_sql('SELECT id, first, last FROM authors',
                 connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Bobby,Gray


In [26]:
#lets make sure we only made one modification
cursor.rowcount

1

In [27]:
#lets remove that author using the DELETE FROM
cursor = cursor.execute('DELETE FROM authors WHERE id=6')
cursor.rowcount

1

In [28]:
#now we check
pd.read_sql('SELECT id, first, last FROM authors',
                 connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [29]:
#let us close out of  our database
connection.close()